In [1]:
import pandas as pd
import numpy as np
import recordlinkage as rl 
from recordlinkage.index import Block

In [2]:
df_frame = pd.read_csv("tbl_listing_info_amsterdam_20200104.csv")

In [3]:
print(df_frame.columns)
print(len(df_frame))

Index(['wsa_listing_id', 'customer_name', 'site', 'listing_id',
       'registration_number', 'house_name', 'city', 'postalcode', 'bedrooms',
       'bathrooms', 'beds', 'person_capacity', 'host_name', 'host_id',
       'host_picture', 'instant_bookable', 'is_new_listing', 'region', 'lat',
       'lng', 'picture_url', 'reviews_count', 'room_type_category',
       'room_type', 'space_type', 'property_type', 'booking_count',
       'host_listings_count_hist', 'kpi_capacity_more_than_4',
       'kpi_capacity_more_than_6', 'kpi_instant_bookable',
       'kpi_host_lists_multiple', 'kpi_host_lists_10_or_more', 'days_active',
       'host_start_year', 'host_start_month', 'reviews_year',
       'reviews_6_months', 'reviews_3_months', 'reviews_1_month', 'rating_avg',
       'bad_ratings_count', 'year_total_available', '2021-06_available_days',
       '2021-07_available_days', '2021-08_available_days',
       '2021-09_available_days', '2021-10_available_days',
       '2021-11_available_days', '2

In [4]:
platforms = set([x for x in df_frame['site']])
print(platforms)

{'Booking', 'VRBO', 'Tripadvisor', 'Expedia', 'AIRBNB'}


In [5]:
print(df_frame['picture_url'].loc[4])
print(df_frame['host_picture'].loc[4])

https://a0.muscache.com/im/pictures/53e16c98-8e9a-4c13-993a-f4e32d044fc7.jpg?aki_policy=large
https://a0.muscache.com/im/pictures/user/81f43f29-7c6c-4105-bca6-ed4dc2137794.jpg?aki_policy=profile_x_medium


In [13]:
def tot_nan_column(df,column):  
    df[column].notna().sum()
    nan_val= df[column].notna().sum()
    return nan_val

In [6]:
df_bnb = df_frame[df_frame['site'] == 'AIRBNB']
df_book = df_frame[df_frame['site'] == 'Booking']
df_exp = df_frame[df_frame['site'] == 'Expedia']
df_trip = df_frame[df_frame['site'] == 'Tripadvisor']
df_verb = df_frame[df_frame['site'] == 'VRBO']

In [7]:
print('AIRBNB total lisitngs : ',len(df_bnb))
print('Booking total lisitngs : ',len(df_book))
print('Expedia total lisitngs : ',len(df_exp))
print('Tripadvisor total lisitngs : ',len(df_trip))
print('VRBO total lisitngs : ',len(df_verb))

AIRBNB total lisitngs :  21173
Booking total lisitngs :  621
Expedia total lisitngs :  168
Tripadvisor total lisitngs :  523
VRBO total lisitngs :  688


In [14]:
print('Total longitude',tot_nan_column(df_frame,'lng'))
print('Total latitude',tot_nan_column(df_frame,'lat'))
print('Total AirBnB longitude',tot_nan_column(df_bnb,'lng'))
print('Total AirBnB latitude',tot_nan_column(df_bnb,'lat'))
print('Total Booking longitude',tot_nan_column(df_book,'lng'))
print('Total Booking latitude',tot_nan_column(df_book,'lat'))
print('Total Expedia longitude',tot_nan_column(df_exp,'lng'))
print('Total Expedia latitude',tot_nan_column(df_exp,'lat'))
print('Total TripAD longitude',tot_nan_column(df_trip,'lng'))
print('Total TripAD latitude',tot_nan_column(df_trip,'lat'))
print('Total VRBO longitude',tot_nan_column(df_verb,'lng'))
print('Total VRBO latitude',tot_nan_column(df_verb,'lat'))

Total longitude 21921
Total latitude 21921
Total AirBnB longitude 19921
Total AirBnB latitude 19921
Total Booking longitude 621
Total Booking latitude 621
Total Expedia longitude 168
Total Expedia latitude 168
Total TripAD longitude 523
Total TripAD latitude 523
Total VRBO longitude 688
Total VRBO latitude 688


In [8]:
df_bnb['postalcode'] = df_bnb['postalcode'].map(lambda x: float("NaN") if x =='Onbekend' else x)
df_bnb['postalcode']

<ipython-input-8-540c2addb53d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bnb['postalcode'] = df_bnb['postalcode'].map(lambda x: float("NaN") if x =='Onbekend' else x)


0            NaN
1            NaN
2            NaN
3            NaN
4        1072 JR
          ...   
21254        NaN
21255        NaN
21256        NaN
21257    1013 SP
21258        NaN
Name: postalcode, Length: 21173, dtype: object

In [9]:
#Calculate non nan values in their columns
def in_nan(site):
       nan_val = {}
       for x in df_frame.columns :   
              if df_frame[df_frame['site'] == site][x].notna().sum() != 0:
                     nan_val[x]= df_frame[df_frame['site'] == site][x].notna().sum()
                     
       return nan_val

In [10]:
#Not nan values in their columns
nan_aribnb = in_nan('AIRBNB')
nan_book = in_nan('Booking')
nan_exp = in_nan('Expedia')
nan_trip = in_nan('Tripadvisor')
nan_vrb = in_nan('VRBO')

In [70]:
nan_trip.keys()

dict_keys(['wsa_listing_id', 'customer_name', 'site', 'listing_id', 'house_name', 'city', 'postalcode', 'bedrooms', 'bathrooms', 'beds', 'person_capacity', 'instant_bookable', 'is_new_listing', 'region', 'lat', 'lng', 'reviews_count', 'kpi_instant_bookable', 'listing_url', 'first_found_date', 'listing_published', 'listing_can_book'])

In [12]:
print('aribnb columns withouth nan values:' , len(nan_aribnb))
print('book columns without nan values:' , len(nan_book))
print('exp columns without nan values:' , len(nan_exp))
print('trip columns without nan values:' , len(nan_trip))
print('vrb columns without nan values:' , len(nan_vrb))

aribnb columns withouth nan values: 62
book columns without nan values: 26
exp columns without nan values: 21
trip columns without nan values: 22
vrb columns without nan values: 26


In [14]:
#indexer = recordlinkage.Index()
indexer = rl.BlockIndex(on='postalcode')
candidate_links = indexer.index(df_bnb, df_verb)

print (len(candidate_links))

1186


In [15]:
compare_cl = rl.Compare()

compare_cl.exact('postalcode', 'postalcode', label='postalcode')
compare_cl.numeric('lat', 'lat', label='lat')
compare_cl.numeric('lng', 'lng', label='lng')


features = compare_cl.compute(candidate_links, df_bnb, df_verb)

In [17]:
features.sum(axis=1).value_counts().sort_index(ascending=False).iloc[:20]

2.999999     1
2.999998     1
2.999998     1
2.999998     1
2.999998     1
2.999997     4
2.999997     1
2.999997    45
2.999997     1
2.999996     1
2.999996     1
2.999995     1
2.999984    11
2.999983    36
2.999982     1
2.999980     1
2.999977     1
2.999977     1
2.999976     1
2.999974     1
dtype: int64

In [30]:
features[features.sum(axis=1) > 2.999982]

postalcode       lat       lng
12125 22515           1  1.000000  0.999998
      22517           1  1.000000  0.999998
12126 22515           1  1.000000  0.999998
      22517           1  1.000000  0.999998
8290  22920           1  0.999999  0.999999
...                 ...       ...       ...
14879 22633           1  0.999997  0.999998
10766 22780           1  0.999998  1.000000
10797 23085           1  1.000000  0.999998
20993 22570           1  0.999999  0.999997
20196 22985           1  0.999999  0.999999

[106 rows x 3 columns]

In [31]:

rl.write_annotation_file(
    "annotation_bnb_vrb.json",
    features[features.sum(axis=1) > 2.999982  ].index,
    df_bnb[['wsa_listing_id', 'customer_name', 'site', 'listing_id', 'house_name', 'city', 'postalcode', 'bedrooms', 'bathrooms', 'beds', 'person_capacity', 'host_name', 'host_picture', 'instant_bookable', 'is_new_listing', 'region', 'lat', 'lng', 'picture_url', 'reviews_count', 'property_type', 'kpi_instant_bookable', 'listing_url', 'first_found_date', 'listing_published', 'listing_can_book']],
    df_verb[['wsa_listing_id', 'customer_name', 'site', 'listing_id', 'house_name', 'city', 'postalcode', 'bedrooms', 'bathrooms', 'beds', 'person_capacity', 'host_name', 'host_picture', 'instant_bookable', 'is_new_listing', 'region', 'lat', 'lng', 'picture_url', 'reviews_count', 'property_type', 'kpi_instant_bookable', 'listing_url', 'first_found_date', 'listing_published', 'listing_can_book']],
    dataset_a_name="AirBnb",
    dataset_b_name="VRBO"
)

In [46]:
df_verb['listing_url'].loc[23085]

'https://www.vrbo.com/nl-nl/vakantiewoning/p8848002?noDates=true'

In [42]:

df_bnb['picture_url'].loc[7959]

'https://a0.muscache.com/im/pictures/cea0b565-370d-4bfe-a80c-b90a155aae9b.jpg?aki_policy=large'

In [50]:
result = rl.read_annotation_file('result_bnb_vrb.json')
print('Matches in AirBnB and VRBO with Postal code Lat, Ln:',len(result.links))
print('Non-matches in AirBnB and VRBO with Postal code Lat, Ln:',len(result.distinct))

Matches in AirBnB and VRBO with Postal code Lat, Ln: 19
Non-matches in AirBnB and VRBO with Postal code Lat, Ln: 87


In [80]:
#indexer = recordlinkage.Index()
indexer = rl.BlockIndex(on='postalcode')
candidate_links = indexer.index(df_bnb, df_trip)

print (len(candidate_links))

783


In [81]:
compare_cl = rl.Compare()

compare_cl.exact('postalcode', 'postalcode', label='postalcode')
compare_cl.numeric('lat', 'lat', label='lat')
compare_cl.numeric('lng', 'lng', label='lng')


features = compare_cl.compute(candidate_links, df_bnb, df_trip)

In [82]:
features.sum(axis=1).value_counts().sort_index(ascending=False).iloc[80:100]

2.999904    1
2.999903    1
2.999903    1
2.999903    2
2.999903    2
2.999902    1
2.999902    1
2.999900    1
2.999900    1
2.999899    1
2.999898    1
2.999898    1
2.999897    2
2.999897    1
2.999896    1
2.999895    1
2.999895    1
2.999893    6
2.999893    1
2.999892    1
dtype: int64

In [83]:
features[features.sum(axis=1) > 2.999892 ]

postalcode       lat       lng
86    22424           1  0.999997  0.999951
293   22009           1  0.999962  0.999945
3681  22009           1  0.999978  0.999970
458   22302           1  0.999995  0.999931
461   22075           1  0.999976  0.999987
...                 ...       ...       ...
18094 22176           1  0.999969  0.999960
      22239           1  0.999969  0.999960
19168 22011           1  0.999997  0.999898
20799 22408           1  0.999994  0.999947
      22421           1  0.999992  0.999951

[129 rows x 3 columns]

In [78]:
print(df_bnb[nan_trip.keys()].loc[20799])
print(df_trip[nan_trip.keys()].loc[22408])

wsa_listing_id                                       15454
customer_name                                    Amsterdam
site                                                AIRBNB
listing_id                                          920560
house_name                                   The Guestroom
city                                             Amsterdam
postalcode                                         1098 CP
bedrooms                                              1,00
bathrooms                                             1,00
beds                                                  2,00
person_capacity                                          4
instant_bookable                                         1
is_new_listing                                           0
region                                      Amsterdam-Oost
lat                                                52.3545
lng                                                4.93366
reviews_count                                          1

In [84]:
rl.write_annotation_file(
    "annotation_bnb_trip.json",
    features[features.sum(axis=1) > 2.999892 ].index,
    df_bnb[nan_trip.keys()],
    df_trip[nan_trip.keys()],
    dataset_a_name="AirBnb",
    dataset_b_name="Trip"
)

In [86]:
df_trip['picture_url'].loc[22009]


nan

In [89]:
df_bnb['picture_url'].loc[743]


'https://a0.muscache.com/im/pictures/ab5d3785-4977-463f-8d85-f46c7d52807d.jpg?aki_policy=large'

In [106]:
result = rl.read_annotation_file('result_bnb_trip.json')
print('Matches in AirBnB and VRBO with Postal code Lat, Ln:',len(result.links))
print('Non-matches in AirBnB and VRBO with Postal code Lat, Ln:',len(result.distinct))

Matches in AirBnB and VRBO with Postal code Lat, Ln: 28
Non-matches in AirBnB and VRBO with Postal code Lat, Ln: 101
